In [ ]:
import sys
import datetime
import mysql.connector
import random
import os
from dotenv import load_dotenv

load_dotenv()

# Establishes a database connection
def connectToDatabase():
    try:
        cnx = mysql.connector.connect(
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'), 
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'))
        cursor = cnx.cursor()
        return cnx, cursor
    except mysql.connector.Error as err:
        print(f"Error connecting to database: {err}")
        sys.exit()

# Creates a new customer account
def createAccount():

    connection, cursor = connectToDatabase()

    print("\nCreating a new customer account!\n")
    fname = input("Enter your first name: ")
    lname = input("Enter your last name: ")
    phone_num = input("Enter your phone number: ")
    email = input("Enter your email: ")
    rewards_pass = input("Enter a password for your account: ")
    zip_code = input("Enter your zip code: ")
    address = input("Enter your address: ")
    city = input("Enter your city: ")
    state = input("Enter your state abbreviation: ")
    country = input("Enter your country: ")

    try:
            # Getting the highest customer id
            cursor.execute("select max(customer_id) from customer") 
            result = cursor.fetchone()
            highest_customer_id = result[0] if result[0] else 0
            new_customer_id = highest_customer_id + 1

            # Getting the highest member id
            cursor.execute("select max(member_id) from members")
            result = cursor.fetchone()
            highest_member_id = result[0] if result[0] else 0 
            new_member_id = highest_member_id + 1

            # Inserting into customer table
            cursor.execute("insert into customer (customer_id) values (%s)", (new_customer_id,))

            # Inserting into members table with customer details
            cursor.execute("insert into members (customer_id, member_id, first_name, last_name, phone_num, email, zip_code, address, city, state, country) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            (new_customer_id, new_member_id, fname, lname, phone_num, email, zip_code, address, city, state, country,),)

            # Getting the highest payment id
            cursor.execute("select max(payment_id) from paymentType")
            result = cursor.fetchone()
            new_payment_id = (result[0] if result[0] else 0) + 1
            cursor.execute("insert into paymentType (payment_id) values (%s)", (new_payment_id,))
    
            print("\n💳 Add a Payment Method")
            card_type = input("Will you use a credit card or gift card? (credit/gift): ").strip().lower()
    
            if card_type == "credit":
                name = input("Name on Card: ")
                number = input("Credit Card Number: ")
                cvv = input("CVV: ")
                expiry = input("Expiration Date (YYYY-MM-DD): ")
                # inserting credit card information
                cursor.execute("""
                    insert into creditInfo (
                        payment_id, name_on_card, credit_card_num, cvv, expiration_date
                    ) values (%s, %s, %s, %s, %s)
                """, (new_payment_id, name, number, cvv, expiry))
            
            elif card_type == "gift":
                number = input("Gift Card Number: ")
                cvv = input("CVV: ")
                balance = float(input("Balance on Card: "))
                # inserting gift card information
                cursor.execute("""
                    insert into giftCardInfo (
                        payment_id, gift_card_no, cvv, balance
                    ) values (%s, %s, %s, %s)
                """, (new_payment_id, number, cvv, balance))
            
            else:
                raise ValueError("Invalid card type. Please enter 'credit' or 'gift'.")
    
            # Linking customer to payment method
            cursor.execute("insert into customerPayment (customer_id, payment_id) values (%s, %s)", (new_customer_id, new_payment_id))

            connection.commit()
            print(f"Account created successfully! Your customer ID is: {new_customer_id}")
    except mysql.connector.Error as err:
        connection.rollback()
        print(f"Error creating account: {err}")
    finally:
        input("\nPress enter to continue.")

# list the avaliable products in the online store
def listAvaliableProducts():
    connection, cursor = connectToDatabase()

    # find all product ids in the system
    try:
        # Fetch all product_ids from the products table
        cursor.execute("SELECT product_id FROM products;")
        results = cursor.fetchall()
    
        # Extract the product_ids into a list
        product_ids = [row[0] for row in results]
        
    except mysql.connector.Error as err:
        print(f"Error: {err}")

    # query to get all values from the database 
    query = ("""
    SELECT DISTINCT TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE 
    FROM information_schema.columns 
    WHERE TABLE_SCHEMA NOT IN ('mysql', 'information_schema', 'performance_schema', 'sys')
    """)
    cursor.execute(query)
    columns = cursor.fetchall()

    for product_id_search in product_ids:
        matching_tables = []
        found = False
        
        for schema, table, column, data_type in columns:
            
            if column == 'product_id' and table not in ('warehouseInventory', 'storeInventory', 'itemsPurchased'):
        
                if isinstance(product_id_search, str): 
                    query = '''
                    SELECT 1 FROM `{}`.`{}` 
                    WHERE `{}` LIKE %s LIMIT 1
                    '''.format(schema, table, column)
                    cursor.execute(query, (f"%{product_id_search}%",))
            
                elif isinstance(product_id_search, int):
                    query = '''
                    SELECT 1 FROM `{}`.`{}` 
                    WHERE `{}` = %s LIMIT 1
                    '''.format(schema, table, column)
                    cursor.execute(query, (product_id_search,))
                
                result = cursor.fetchone()
                
                if result:
                    matching_tables.append(table)
                    found = True
    
        if not found:
            print("No tables in 'zaradatabase' contain product_id = specified value.")
            continue
    
        if len(matching_tables) < 2:
            continue
            
        # Build FROM and JOIN clause dynamically
        base_table = matching_tables[0]
        join_clause = f"FROM zaradatabase.`{base_table}` AS t0\n"
    
        for i, table in enumerate(matching_tables[1:], start=1):
            join_clause += f"JOIN zaradatabase.`{table}` AS t{i} USING (product_id)\n"
    
        select_clause = "SELECT *\n"
        final_query = select_clause + join_clause + "WHERE t0.product_id = %s"
    
        cursor.execute(final_query, (product_id_search,))
        results = cursor.fetchall()

        if results:

            # Get column names from cursor description immediately after executing the relevant query
            headers = [desc[0] for desc in cursor.description]
        
            # Convert rows to dictionaries
            rows_as_dicts = [dict(zip(headers, row)) for row in results]
        
            # Determine column widths
            col_widths = [len(h) for h in headers]
            for row in rows_as_dicts:
                for i, value in enumerate(row.values()):
                    col_widths[i] = max(col_widths[i], len(str(value)))
        
            # Print header row
            header_row = " | ".join(h.ljust(col_widths[i]) for i, h in enumerate(headers))
            print(header_row)
            print("-" * len(header_row))
        
            # Print each data row
            for row in rows_as_dicts:
                formatted_row = " | ".join(str(row[h]).ljust(col_widths[i]) for i, h in enumerate(headers))
                print(formatted_row)

            # also print quantity            
            quantityquery = (f'''select quantity from storeInventory where product_id = {product_id_search} 
            and store_id = 1''')

            cursor.execute(quantityquery)
            quantityresults = cursor.fetchall()

            headers = [desc[0] for desc in cursor.description]
        
            # Convert rows to dictionaries
            rows_as_dicts = [dict(zip(headers, row)) for row in quantityresults]
        
            # Determine column widths
            col_widths = [len(h) for h in headers]
            for row in rows_as_dicts:
                for i, value in enumerate(row.values()):
                    col_widths[i] = max(col_widths[i], len(str(value)))
        
            # Print header row
            header_row = " | ".join(h.ljust(col_widths[i]) for i, h in enumerate(headers))
            print(header_row)
            print("-" * len(header_row))
        
            # Print each data row
            for row in rows_as_dicts:
                formatted_row = " | ".join(str(row[h]).ljust(col_widths[i]) for i, h in enumerate(headers))
                print(formatted_row)
            
            print("_" * 100, "\n")

        else:
            print("No results found.")
            product_data[product_id_search] = []


# Places an order for a member
def placeOrderMember(customer_id=None):
    store_id = 1
    connection, cursor = connectToDatabase()
    
    if connection is None:
        return 

    try:
        order_items = []
        order_date = datetime.datetime.now()
        # Getting the highest order id
        cursor.execute("select max(order_id) from orders")
        result = cursor.fetchone()
        new_order_id = result[0] if result[0] else 0
        new_order_id += 1
        tracking_num = random.randint(111111111111, 999999999999)
        return_status = None
        date_of_return = None
        total_price = 0.0

        while True:
            # showing the avalaible products
            listAvaliableProducts()
            
            product_id = int(input("\nEnter the product ID: "))
            quantity = int(input("Enter the quantity: "))
            order_items.append((product_id, quantity))

            more = input("Do you want to add another product to the order? (yes/no): ").strip().lower()
            if more != "yes":
                break

        # Creating a new order
        cursor.execute('''insert into orders (order_id, order_date, tracking_num) values (%s, %s, %s)''',
                        (new_order_id, order_date, tracking_num),)

        for product_id, quantity in order_items:
            # Checking if the product exists
            cursor.execute("select price from products where product_id = %s", (product_id,))
            product_price_result = cursor.fetchone()
            if not product_price_result:
                connection.rollback()  # Rollback if product doesn't exist
                raise ValueError(f"Product with ID {product_id} not found")
            product_price = product_price_result[0]

            # Finding a store with enough quantity
            cursor.execute(
                '''select store_id, quantity from storeInventory 
                   where product_id = %s and quantity >= %s 
                   order by quantity desc limit 1''',
                (product_id, quantity),)
            store_result = cursor.fetchone()
            if not store_result:
                connection.rollback()
                raise ValueError(f"Not enough stock of product {product_id} in store id {store_id} to fulfill the order.")
            store_id, available_quantity = store_result

            # Fetching percentage on sale for the selected store
            cursor.execute(
                "select percentage_on_sale from storeInventory where product_id = %s and store_id = %s",
                (product_id, store_id),)
            sale_result = cursor.fetchone()
            percentage_on_sale = sale_result[0] if sale_result else 0

            # Calculating price with discount
            price_at_purchase = product_price * (1 - percentage_on_sale)
            price_at_purchase = round(price_at_purchase, 2)
            
            # Inserting into itemsPurchased
            cursor.execute('''insert into itemsPurchased (order_id, product_id, quantity, price_at_purchase, return_status, date_of_return) values (%s, %s, %s, %s, %s, %s)''',
                            (new_order_id, product_id, quantity, price_at_purchase, return_status, date_of_return))

            # updating the inventory function
            updateInventory(product_id, quantity)

            # Calling check and reorder inventory to determine if the order that was just placed ran inventory too low
            checkAndReorderInventory(product_id) 
            
        # Adding to customerOrder table
        if customer_id:
            cursor.execute('''insert into customerOrder (customer_id, order_id) values (%s, %s)''', (customer_id, new_order_id))

        # Recording which store fulfilled the order
        cursor.execute('''insert into storeOrderedFrom (order_id, store_id) values (%s, %s)''', (new_order_id, store_id))

        print("\nThese are the payment methods we have on file for your account:")
        # Getting payment methods for the customer
        cursor.execute(f"select payment_id from customerPayment where customer_id = '{customer_id}';")
        payment_results = cursor.fetchall()
        if payment_results:
            # Getting credit card information
            cursor.execute(f"select payment_id, credit_card_num from customerPayment join creditinfo using (payment_id) where customer_id = '{customer_id}';")
            cc_results = cursor.fetchall()
            if cc_results:
                print("\n Credit Cards:")
                print("{:<12} {:<20}".format("Payment ID", "Card Number"))
                print("-" * 35)
                for row in cc_results:
                    payment_id, credit_card_num = row
                    print("{:<12} {:<20}".format(payment_id, credit_card_num))
            # Getting gift card information
            cursor.execute(f"select payment_id, gift_card_no from customerPayment join giftCardInfo using (payment_id) where customer_id = '{customer_id}';")
            gc_results = cursor.fetchall()
            if gc_results:
                print("\n Gift Cards on File:")
                print("{:<12} {:<20}".format("Payment ID", "Gift Card Number"))
                print("-" * 35)
                for result in gc_results:
                    payment_id, gift_card_no = result
                    print("{:<12} {:<20}".format(payment_id, gift_card_no))

        else:
            print(f"No payment found for customer ID: {customer_id}")

        if len(payment_results) > 1:
            # Asking the user for payment choice
            paymentChoice = input("\nPlease input what payment method you would like to use (enter payment_id): ")
            paymentChoice = int(paymentChoice)

        elif len(payment_results) == 1:
            # Automatically selecting the only payment method
            cursor.execute(f"select payment_id from customerPayment where customer_id = '{customer_id}';")
            paymentChoice = cursor.fetchone()[0]

        # Recording payment method for order
        cursor.execute('''insert into orderPayment (order_id, payment_id) values (%s, %s)''', (new_order_id, paymentChoice))

        connection.commit()
        print("\nOrder placed successfully!")
        print(f"\n🧾 Your order ID is: {new_order_id}")
        input("\nPress Enter to continue.")
    except mysql.connector.Error as err:
        connection.rollback()
        print(f"Error placing order: {err}")
    except ValueError as ve:
        connection.rollback()
        print(f"Error: {ve}")
    finally:
        cursor.close()
        connection.close()

# Placing an order as a guest
def placeOrderGuest(customer_id=None):
  
    connection, cursor = connectToDatabase()

    store_id = 1

    if connection is None:
        # Exiting if database connection failed
        return

    try:
        print("What products would you like to add to your shopping cart?")
        order_items = []
        # Getting the highest order id
        cursor.execute("select max(order_id) from orders")
        new_order_id = (cursor.fetchone()[0] or 0) + 1
        order_date = datetime.datetime.now()
        tracking_num = random.randint(111111111111, 999999999999)

        while True:
            
            listAvaliableProducts()
            
            product_id = int(input("Enter product ID: "))
            quantity = int(input("Enter quantity: "))
            order_items.append((product_id, quantity))
            more = input("\nAdd another product? (yes/no): ").strip().lower()
            if more != "yes":
                break

        total_price = 0
        print("\nYour Shopping Cart:")
        print("-" * 40)
        print("{:<10} {:<5} {:<10} {:<10}".format("Product", "Qty", "Price", "Total"))
        print("-" * 40)

        for product_id, quantity in order_items:  # Iterate through order items
            # Fetching product price
            cursor.execute("select price from products where product_id = %s", (product_id,))
            product_price_result = cursor.fetchone()
            if not product_price_result:
                connection.rollback()
                raise ValueError(f"Product with ID {product_id} not found")
            product_price = product_price_result[0]

            # Finding a store with enough quantity
            cursor.execute(
                '''select store_id, quantity from storeInventory
                   where product_id = %s and quantity >= %s
                   order by quantity desc limit 1''',
                (product_id, quantity),)
            store_result = cursor.fetchone()
            if not store_result:
                connection.rollback()
                raise ValueError(
                    f"Not enough stock of product {product_id} in any store to fulfill the order.")
            store_id, available_quantity = store_result

            # Fetching percentage on sale for the selected store
            cursor.execute(
                "select percentage_on_sale from storeInventory where product_id = %s and store_id = %s",
                (product_id, store_id),)
            sale_result = cursor.fetchone()
            percentage_on_sale = sale_result[0] if sale_result else 0

            # Calculating price with discount
            price_at_purchase = round(product_price * (1 - percentage_on_sale), 2)

            # Getting product name for display in cart
            cursor.execute("select product_name from products where product_id = %s", (product_id,))
            product_name_result = cursor.fetchone()
            product_name = product_name_result[0] if product_name_result else "Unknown Product"
            item_total_price = price_at_purchase * quantity
            total_price += item_total_price
            print("{:<10} {:<5} ${:<9.2f} ${:<10.2f}".format(product_name, quantity, price_at_purchase, item_total_price))

        print("\nPlacing an order as a guest.\n")
        fname = input("Enter your first name: ")
        lname = input("Enter your last name: ")
        phone_num = input("Enter your phone number: ")
        email = input("Enter your email: ")
        address = input("Enter your address: ")
        zip_code = input("Enter your zip code: ")
        city = input("Enter your city: ")
        state = input("Enter your state abbreviation: ")
        country = input("Enter your country: ")

        # Getting next customer ID and adding to customer + nonMember tables
        cursor.execute("select max(customer_id) from customer")
        customer_id = (cursor.fetchone()[0] or 0) + 1
        cursor.execute("insert into customer (customer_id) values (%s)", (customer_id,))
        cursor.execute("insert into nonMember (customer_id) values (%s)", (customer_id,))
        
        # Adding guest customer information
        cursor.execute(
            '''insert into onlineNM
                (customer_id, first_name, last_name, phone_num, email, zip_code, address, city, state, country)
                values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
            (customer_id, fname, lname, phone_num, email, zip_code, address, city, state, country))

        # Creating a new order
        cursor.execute("insert into orders (order_id, order_date, tracking_num) values (%s, %s, %s)",
                        (new_order_id, order_date, tracking_num))

        for product_id, quantity in order_items:
            # Inserting into itemsPurchased
            cursor.execute('''insert into itemsPurchased
                            (order_id, product_id, quantity, price_at_purchase, return_status, date_of_return)
                            values (%s, %s, %s, %s, null, null)''',
                            (new_order_id, product_id, quantity, price_at_purchase))

            # updating the inventory function
            updateInventory(product_id, quantity)
            
            # Determining if their order brought the inventory too low
            checkAndReorderInventory(product_id)

        # Linking customer to order
        cursor.execute("insert into customerOrder (customer_id, order_id) values (%s, %s)", (customer_id, new_order_id))
        
        # Recording which store fulfilled the order
        cursor.execute("insert into storeOrderedFrom (order_id, store_id) values (%s, %s)", (new_order_id, store_id))

        # Creating payment type
        cursor.execute("select max(payment_id) from paymentType")
        new_payment_id = (cursor.fetchone()[0] or 0) + 1
        cursor.execute("insert into paymentType (payment_id) values (%s)", (new_payment_id,))

        payment_type = input("\nAre you using a credit card or gift card? (credit/gift): ").lower().strip()
        if payment_type == "credit":
            # Adding credit card information
            card_name = input("Name on card: ")
            number = input("Credit card number: ")
            cvv = input("CVV: ")
            expiry = input("Expiration date (YYYY-MM-DD): ")
            cursor.execute('''insert into creditInfo (payment_id, name_on_card, credit_card_num, cvv, expiration_date)
                                        values (%s, %s, %s, %s, %s)''',
                            (new_payment_id, card_name, number, cvv, expiry))
        elif payment_type == "gift":
            # Adding gift card information
            number = input("Gift card number: ")
            cvv = input("CVV: ")
            balance = float(input("Balance: "))
            cursor.execute('''insert into giftCardInfo (payment_id, gift_card_no, cvv, balance)
                                        values (%s, %s, %s, %s)''',
                            (new_payment_id, number, cvv, balance))
        else:
            raise ValueError("\nInvalid payment type.")

        # Linking customer to payment and order to payment
        cursor.execute("insert into customerPayment (customer_id, payment_id) values (%s, %s)", (customer_id, new_payment_id))
        cursor.execute("insert into orderPayment (order_id, payment_id) values (%s, %s)", (new_order_id, new_payment_id))

        connection.commit()
        print(f"Guest order placed successfully! Your order ID is {new_order_id}.")
        input("Press Enter to continue...")

    except mysql.connector.Error as err:
        connection.rollback()
        print(f"MySQL Error: {err}")
    except ValueError as ve:
        connection.rollback()
        print(f"Validation Error: {ve}")
    finally:
        cursor.close()
        connection.close()


def checkAndReorderInventory(product_id):

    reorder_threshold = 20
    transfer_quantity = 100
    
    connection, cursor = connectToDatabase()

    try:
        # find the quantity of the product using product_id
        cursor.execute(
            f'''select quantity from storeInventory 
            where store_id = 1 
            and product_id = {product_id}; ''')
        
        currentquantity = cursor.fetchone()
        
        if not currentquantity:
            print(f"Online store does not stock product {product_id}.")
            return
            
        else: #if there is a current quantity value 
            store_quantity = currentquantity[0] # extract the integer

        # Only reorder if the quantity is below or equal to the threshold
        if store_quantity > reorder_threshold:
            return  # No need to reorder if we have enough stock

        elif store_quantity <= reorder_threshold:  # if item does need to be reordered
            
            # find the warehouse that supplies the online store 
            cursor.execute(
                f'''select warehouse_id 
                from supplies 
                where store_id = 1;''')
            
            warehouseresult = cursor.fetchone()
    
            if not warehouseresult:
                print(f'''Warehouse supplying online store does not 
                have enough stock of product {product_id} to restock online store.''')
                return

            else: # if warehouse result 

                warehouse_result = warehouseresult[0]   # extract integer 

                # query to get the quantity of the item that warehouse supplying online store has
                cursor.execute(
                    f'''select quantity
                    from warehouseInventory 
                    where warehouse_id = {warehouse_result}
                    and product_id = {product_id};''')
                
                warehouse_item_quantity = cursor.fetchone()
                warehouseItemQuantity = warehouse_item_quantity[0]  # extract the integer

                # amount to transfer = 100
                # remove transfer amount from warehouse 
                new_warehouse_quantity = warehouseItemQuantity - transfer_quantity
                cursor.execute(f'''UPDATE warehouseInventory 
                    SET quantity = {new_warehouse_quantity} 
                    WHERE warehouse_id = {warehouse_result} and product_id = {product_id};''')

                # add transfer amount to store inventory 
                new_store_quantity = store_quantity + 100 
                cursor.execute(f'''UPDATE storeInventory 
                    SET quantity = {new_store_quantity} 
                    WHERE store_id = 1 and product_id = {product_id};''')
        
                print(f"Reordered {transfer_quantity} of product {product_id} from warehouse {warehouse_result} to online store.")
        
                connection.commit()

    except mysql.connector.Error as err:
        connection.rollback()
        print(f"MySQL Error during reorder: {err}")
    finally:
        cursor.close()
        connection.close()
        
# Viewing order details
def viewOrderDetails(customer_id):
    connection, cursor = connectToDatabase()
    print(f"\nViewing order details for customer ID: {customer_id}.")
    
    # Query to get order details
    orderDetailsQuery = '''
    select p.product_name, ip.price_at_purchase, ip.quantity, ip.return_status, ip.date_of_return
    from itemsPurchased as ip join products as p on(ip.product_id = p.product_id)
    where order_id = %s
    '''

    order_id = input("\nWhat order number would you like details on: ")
    idValue = (order_id,)
    try:
        cursor.execute(orderDetailsQuery, idValue)
        
        prod_list = []
        print(f"\nOrder Details for order ID: {order_id}\n")
        print("{:<20} {:<15} {:<10} {:<15} {:<20}".format("Product Name", "Price Paid for Item", "Quantity", "Return Status", "Date of Return"))
        print("-" * 80)
        for product_name, price_at_purchase, quantity, return_status, date_of_return in cursor:
            print("{:<20} ${:<14.2f} {:<10} {:<15} {:<20}".format(product_name, price_at_purchase, quantity, str(return_status), str(date_of_return)))
            prod_list.append(product_name)
        input("\nPress Enter to continue.\n")
    except mysql.connector.Error as err:
        print(f"Error viewing order details: {err}")
    finally:
        cursor.close()
        connection.close()

# Adding a payment method for a member
def addPaymentMethod(customer_id):
    connection, cursor = connectToDatabase()
    print(f"\nAdding a new payment method for customer ID: {customer_id}.\n")
    try:
        # Getting the highest payment id
        cursor.execute("select max(payment_id) from paymentType")
        result = cursor.fetchone()
        highest_payment_id = result[0] if result[0] else 0 
        new_payment_id = highest_payment_id + 1
        cardType = input("Do you want to add a credit card or a gift card? Enter either 'credit' or 'gift': ")
        if cardType.lower() == "credit":
            # Asking for credit card information
            name_on_card = input("Enter the name on the credit card: ")
            credit_card_num = input("Enter the credit card number: ")
            cvv = input("Enter the cvv: ")
            expiration_date = input("Enter the expiration date (YYYY-MM-DD): ")
            
            # Creating payment type and adding credit card info
            cursor.execute("insert into paymentType (payment_id) values (%s)", (new_payment_id,))
            cursor.execute("insert into creditInfo (payment_id, name_on_card, credit_card_num, cvv, expiration_date) values (%s, %s, %s,%s, %s)", 
                           (new_payment_id, name_on_card, credit_card_num, cvv, expiration_date))
            
            # Linking customer to payment
            cursor.execute("insert into customerPayment (customer_id, payment_id) values (%s, %s)", (customer_id, new_payment_id))
        elif cardType.lower() == "gift":
            # Asking for gift card information
            gift_card_no = input("Enter the gift card number: ")
            cvv = input("Enter the cvv: ")
            balance = input("Enter the balance on the card: ")
            
            # Creating payment type and adding gift card info
            cursor.execute("insert into paymentType (payment_id) values (%s)", (new_payment_id,))
            cursor.execute("insert into giftCardInfo (payment_id, gift_card_no, cvv, balance) values (%s, %s, %s, %s)", 
                           (new_payment_id, gift_card_no, cvv, balance))
            
            # Linking customer to payment
            cursor.execute("insert into customerPayment (customer_id, payment_id) values (%s, %s)", (customer_id, new_payment_id))
        connection.commit()
        print("\nPayment added!\n")
    except mysql.connector.Error as err:
        connection.rollback()
        print(f"Error adding payment method: {err}")
    finally:
        cursor.close()
        connection.close()
    input("\nPress Enter to continue.\n")

# Showing personal account information for members
def showPersonalInformation(customer_id):
    print(f"\nCurrent personal information for customer ID: {customer_id}.")
    connection, cursor = connectToDatabase()

    # Query to get personal information
    personalInfoQuery = '''select first_name, last_name, phone_num, email, zip_code, address, city, state, country from members where customer_id = %s'''

    idValue = (customer_id,)
    try:
        cursor.execute(personalInfoQuery, idValue)
        result = cursor.fetchone()  
        if result:
            first_name, last_name, phone_num, email, zip_code, address, city, state, country = result
            print(f"\nCurrent personal information for customer ID: {customer_id}.")
            print('First Name\tLast Name\tPhone Number\tEmail\tZip Code\tAddress\tCity\tState\tCountry')
            print(f"{first_name}\t{last_name}\t{phone_num}\t{email}\t{zip_code}\t{address}\t{city}\t{state}\t{country}")
        else:
            print(f"No personal information found for customer ID: {customer_id}")
        input("Press Enter to continue.")
    except mysql.connector.Error as err:
        print(f"Error viewing personal information: {err}")
    finally:
        cursor.close()
        connection.close()

# Changing personal account information for members
def changePersonalInformation(customer_id):
    connection, cursor = connectToDatabase()
    print(f"\nChanging personal information for customer ID: {customer_id}.")

    try:
        # Fetching current information
        cursor.execute('''select first_name, last_name, phone_num, email, zip_code, address, city, state, country 
                          from members where customer_id = %s''', (customer_id,))
        result = cursor.fetchone()

        if not result:
            print("No personal information found.")
            return

        # Showing current info
        fields = ['first_name', 'last_name', 'phone_num', 'email', 'zip_code', 'address', 'city', 'state', 'country']
        current_info = dict(zip(fields, result))
        print("\nCurrent Personal Information:")
        for k, v in current_info.items():
            print(f"{k.replace('_', ' ').title()}: {v}")

        # Asking which fields to change
        print("\nWhich fields would you like to change? (Enter comma-separated field names)")
        print("Options: " + ", ".join(fields))
        to_change = input("Your choice: ").strip().lower().split(',')

        # Cleaning and validating
        to_change = [field.strip() for field in to_change if field.strip() in fields]
        if not to_change:
            print("No valid fields selected.")
            return

        # Getting new values
        updated_info = current_info.copy()
        for field in to_change:
            updated_info[field] = input(f"Enter new value for {field.replace('_', ' ').title()}: ")

        # Showing before and after
        print("\nReview your changes:")
        print("{:<20} {:<30} {:<30}".format("Field", "Current", "Updated"))
        print("-" * 80)
        for field in fields:
            print("{:<20} {:<30} {:<30}".format(field.replace('_', ' ').title(), str(current_info[field]), str(updated_info[field])))

        confirm = input("\nDo you still want to save these changes? (yes/no): ").strip().lower()
        if confirm != "yes":
            print("No changes were made.")
            return

        # Updating the database
        cursor.execute('''
            update members set first_name=%s, last_name=%s, phone_num=%s, email=%s, 
            zip_code=%s, address=%s, city=%s, state=%s, country=%s where customer_id = %s
        ''', (
            updated_info['first_name'], updated_info['last_name'], updated_info['phone_num'],
            updated_info['email'], updated_info['zip_code'], updated_info['address'],
            updated_info['city'], updated_info['state'], updated_info['country'], customer_id))

        connection.commit()
        print("Personal information updated successfully!")

    except mysql.connector.Error as err:
        connection.rollback()
        print(f"Error updating personal information: {err}")
    finally:
        cursor.close()
        connection.close()

# Adding a new product to a warehouse and a store
def addProductToWarehouseAndStore():
    connection, cursor = connectToDatabase()

    # Getting product information from the user
    product_id = int(input("\nEnter the product ID: "))
    # Check if product already exists
    cursor.execute('select product_id from products where product_id = %s', (product_id,))
    if cursor.fetchone():
        print(f"Error: Product with ID {product_id} already exists.")
        cursor.close()
        connection.close()
        return
    else:
        # If product doesn't exist, get the highest product ID and increment by 1
        cursor.execute("select max(product_id) from products")
        result = cursor.fetchone()
        highest_product_id = result[0] if result[0] else 0 
        new_product_id = highest_product_id + 1
    
    # Get additional product details from user
    price = float(input("Enter the product price: "))
    product_category = input("Enter the product category (Womens/Mens/Babys/Girls/Boys): ")
    product_subcategory = input("Enter the product subcategory (Top/Bottom/Shoes/Accessory/Outwear/Set/Dress/Jumpsuit/Suit/Swimwear): ")
    store_id = int(input("Enter the store ID of the store that you want the product to be at: "))
    initial_quantity = int(input("Enter the initial quantity you want at that store: "))
    percentage_on_sale = float(input("Enter the percentage on sale (e.g., 0 for none, 0.1 for 10% off): "))

    # Create the product name by combining category and subcategory
    product_name = product_category + " " + product_subcategory

    try:
        # Insert into products table
        cursor.execute('''
            insert into products (product_id, product_name, price, product_category) 
            values (%s, %s, %s, %s)
        ''', (new_product_id, product_name, price, product_category))

        # Insert into store inventory
        cursor.execute('''
            insert into storeInventory (store_id, product_id, quantity, percentage_on_sale) 
            values (%s, %s, %s, %s)
        ''', (store_id, new_product_id, initial_quantity, percentage_on_sale))

        # Get all warehouse IDs
        cursor.execute('select warehouse_id from warehouse')
        all_warehouses = cursor.fetchall()

        # Insert into each warehouse's inventory with default quantity of 100
        for (warehouse_id,) in all_warehouses:
            cursor.execute('''
                insert into warehouseInventory (warehouse_id, product_id, quantity) 
                values (%s, %s, 100)
            ''', (warehouse_id, new_product_id))

        # Commit the transaction
        connection.commit()
        print(f"Product '{product_name}' (ID: {product_id}) added with quantity 100 to each warehouse and quantity {initial_quantity} to store ID {store_id}.")

    except mysql.connector.Error as err:
        print(f"Error adding product to inventory: {err}")
        connection.rollback() # Rollback on error
    finally:
        # Close database connections
        cursor.close()
        connection.close()

# Adding inventory to the warehouse
def addInventoryToWarehouse():
    connection, cursor = connectToDatabase()
    print("\nAdding inventory to the warehouse.")
    
    # Get warehouse inventory details from user
    product_id = input("Enter product ID: ")
    warehouse_id = input("Enter warehouse ID: ")
    quantity_to_add = int(input("Enter quantity to add: "))
    
    try:
        # Update warehouse inventory with the new quantity
        cursor.execute("update warehouseInventory set quantity = %s where product_id = %s and warehouse_id = %s", 
                      (quantity_to_add, product_id, warehouse_id))
        connection.commit()
        print("\nInventory added to warehouse\n")
    except mysql.connector.Error as err:
        connection.rollback()
        print(f"Error adding inventory to warehouse: {err}")
    finally:
        cursor.close()
        connection.close()
    input("Press Enter to continue.")

# Adds inventory to a store
def addInventoryToStore():
    connection, cursor = connectToDatabase()

    print("\nAdding inventory to the store.\n")
    
    # Get store inventory details from user
    product_id = input("Enter product ID: ")
    store_id = input("Enter store ID: ")
    quantity_to_add = int(input("Enter quantity to add: "))
    percentage_on_sale = input("Will this item be on sale (if no type 0, if yes type a decimal to represent the percent off): ")
    
    try:
        # Update store inventory with new quantity and sale percentage
        cursor.execute("update storeInventory set quantity = %s, percentage_on_sale = %s where product_id = %s and store_id = %s", 
                      (quantity_to_add, percentage_on_sale, product_id, store_id))
        connection.commit()
        print("\nInventory added to store\n")
    except mysql.connector.Error as err:
        connection.rollback()
        print(f"Error adding inventory to store: {err}")
    finally:
        cursor.close()
        connection.close()
    input("Press Enter to continue.")

# Displaying trends for a specific store
def viewStoreTrends(store_id):
    connection, cursor = connectToDatabase()

    # Get the specific store ID from user
    store_id = int(input("Enter the store ID: ")) 
    print(f"\nDisplaying the top 20 selling products for store ID: {store_id}.")

    # Query to find top selling products for a specific store
    topSellingProductForStoreQuery = '''select p.product_id, p.product_name, sum(ip.quantity) as totalQuantitySold  
    from storeOrderedFrom as s join orders as o on(s.order_id = o.order_id) 
    join itemsPurchased as ip on(o.order_id = ip.order_id) 
    join products as p on(ip.product_id = p.product_id)
    where s.store_id = %s  
    group by p.product_id, p.product_name  
    order by totalQuantitySold desc
    limit 20;'''
    
    idValue = (store_id,)
    try:
        cursor.execute(topSellingProductForStoreQuery, idValue)
        
        # Format and display the results
        print("{:<12} {:<30} {:<20}".format("Product ID", "Product Name", "Total Quantity Sold"))
        print("-" * 70)
        results = cursor.fetchall() 
        for product_id, product_name, totalQuantitySold in results:
            print("{:<12} {:<30} {:<20}".format(product_id, product_name, totalQuantitySold))       
        input("Press Enter to continue.")
    except mysql.connector.Error as err:
        print(f"Error viewing store trends: {err}")  
    finally:
        cursor.close()
        connection.close()

# Displaying trends from all Zara stores
def viewZaraTrends():
    connection, cursor = connectToDatabase()
    print("\nDisplaying overall Zara trends.")
    
    # Menu loop for Zara trends
    while True:
        displayZaraTrendsMenu()
        zara_choice = getMenuChoice(7)
        if zara_choice == 7:
            print("Exiting the application. Thank you!")
            sys.exit()
        elif zara_choice == 6:
            break
        elif zara_choice == 1:
            displayTop20SellingProductsOverall()
        elif zara_choice == 2:
            displayTop20SellingProductsByState()
        elif zara_choice == 3:
            displayStoresWithMostSalesThisYear()
        elif zara_choice == 4:
            displayProductsSoldTogether()
        elif zara_choice == 5:
            displayProductsOutselling()

# Displaying the top 20 selling products overall 
def displayTop20SellingProductsOverall():
    connection, cursor = connectToDatabase()

    # Query to find top 20 selling products across all stores
    query = '''select p.product_id, p.product_name, sum(ip.quantity) as total_quantity_sold
    from products as p join itemsPurchased as ip on(p.product_id = ip.product_id)
    group by p.product_id, p.product_name
    order by total_quantity_sold desc
    limit 20;'''
    
    try:
        cursor.execute(query)
        
        # Format and display the results
        print("\nTop 20 Selling Products Overall")
        print("{:<12} {:<30} {:<20}".format("Product ID", "Product Name", "Total Quantity Sold"))
        print("-" * 70)        
        results = cursor.fetchall()
        for product_id, product_name, total_quantity_sold in results:
            print("{:<12} {:<30} {:<20}".format(product_id, product_name, total_quantity_sold))
    except mysql.connector.Error as err:
        print(f"Error displaying top 20 selling products: {err}")
    finally:
        cursor.close()
        connection.close()
    input("\nPress Enter to continue.\n")


# Displaying the top 20 selling products by state
def displayTop20SellingProductsByState():
    connection, cursor = connectToDatabase()
    
    # Query to find top selling products grouped by state
    query = '''select m.state, p.product_id, p.product_name, sum(ip.quantity) as total_quantity_sold
    from members as m join customer as c on m.customer_id = c.customer_id 
    join orders as o on c.customer_id = o.order_id 
    join itemsPurchased as ip on o.order_id = ip.order_id 
    join products as p on ip.product_id = p.product_id
    group by m.state, p.product_id, p.product_name
    order by m.state, total_quantity_sold desc
    limit 20;'''
    
    try:
        cursor.execute(query)
        
        # Format and display the results
        print("\nTop 20 Selling Products by State")
        print("{:<10} {:<12} {:<30} {:<20}".format("State", "Product ID", "Product Name", "Total Quantity Sold"))
        print("-" * 75)        
        results = cursor.fetchall()
        for state, product_id, product_name, total_quantity_sold in results:
            print("{:<10} {:<12} {:<30} {:<20}".format(state, product_id, product_name, total_quantity_sold))
    except mysql.connector.Error as err:
        print(f"Error displaying top 20 selling products by state: {err}")
    finally:
        cursor.close()
        connection.close()
    input("\nPress Enter to continue.\n")
    

# Displaying the stores with the most sales so far this year
def displayStoresWithMostSalesThisYear():
    connection, cursor = connectToDatabase()

    print("\nDisplaying stores with the most sales this year...")
    
    # Query to find stores with highest sales in the current year
    top_selling_store_by_sales_query = '''select s.store_id, sum(ip.price_at_purchase * ip.quantity) as total_sales
    from storeOrderedFrom as s join orders as o on(s.order_id = o.order_id) 
    join itemsPurchased as ip on(o.order_id = ip.order_id) 
    join products as p on(ip.product_id = p.product_id)
    where year(o.order_date) = year(curdate()) 
    group by s.store_id 
    order by total_sales desc 
    limit 5;
    '''
    
    try:
        cursor.execute(top_selling_store_by_sales_query)
        
        # Format and display the results
        print("\nTop 5 Stores with Highest Sales This Year")
        print("{:<12} {:<20}".format("Store ID", "Total Sales"))
        print("-" * 32)
        for (store_id, total_sales) in cursor:
            print("{:<12} ${:<,.2f}".format(store_id, total_sales))
        input("Press Enter to continue.")
    except mysql.connector.Error as err:
        print(f"Error displaying stores with most sales: {err}")
    finally:
        cursor.close()
        connection.close()

# Displaying products that are often sold together
def displayProductsSoldTogether():
    connection, cursor = connectToDatabase()

    # Get product ID from user
    input_product_id = int(input("\nEnter the product ID: \n"))

    print("\nDisplaying products that customers buy in addition to another product.")

    # Get the product name for the given product ID
    cursor.execute("select product_name from products where product_id = %s", (input_product_id,))
    product_name_result = cursor.fetchone()

    if not product_name_result:
        print(f"Error: Product with ID '{input_product_id}' not found.")
        cursor.close()
        connection.close()
        # Exiting if product id is invalid
        return 

    input_product_name = product_name_result[0]

    # Query to find products frequently bought with the specified product
    query = """
    with p1_orders as (
        select distinct ip.order_id
        from itemsPurchased ip
        where ip.product_id = %s 
    )
    select p.product_name, count(*) as times_bought_with_p1
    from itemsPurchased ip
    join products p on ip.product_id = p.product_id
    where ip.order_id in (select order_id from p1_orders)
        and ip.product_id != %s 
    group by p.product_name
    order by times_bought_with_p1 desc
    limit 3;
    """

    cursor.execute(query, (input_product_id, input_product_id)) 
    results = cursor.fetchall()

    # Format and display the results
    print(f"\nProducts Most Often Bought with '{input_product_name}'")
    print("{:<30} {:<30}".format("Product Name", "Times Bought Together"))
    print("-" * 60)

    if results:
        for product_name, times_bought in results:
            print("{:<30} {:<30}".format(product_name, times_bought))
    else:
        print("No other products were found to be frequently bought together.")

    cursor.close()
    connection.close()

# Comparing which product outsells the other across different stores
def displayProductsOutselling():
    connection, cursor = connectToDatabase()

    print("\nComparing sales of two products across stores.")
    
    # Get two product IDs to compare
    product_id1 = int(input("Enter the first product ID: "))
    product_id2 = int(input("Enter the second product ID: "))

    # Fetch product names for the given IDs
    cursor.execute("select product_name from products where product_id = %s", (product_id1,))
    product_name1_result = cursor.fetchone()
    cursor.execute("select product_name from products where product_id = %s", (product_id2,))
    product_name2_result = cursor.fetchone()

    if not product_name1_result or not product_name2_result:
        print("Error: One or both product IDs are invalid.")
        cursor.close()
        connection.close()
        return

    product_name1 = product_name1_result[0]
    product_name2 = product_name2_result[0]

    # Complex query to compare product sales across stores
    query = """
    with sales_by_store as (
        select so.store_id, p.product_id, p.product_name, sum(ip.price_at_purchase * ip.quantity) as total_sales
        from itemsPurchased ip
        join products p on ip.product_id = p.product_id
        join storeOrderedFrom so on ip.order_id = so.order_id
        where p.product_id in (%s, %s) 
        group by so.store_id, p.product_id, p.product_name
    ),
    sales_comparison as (
        select store_id, max(case when product_id = %s then total_sales else 0 end) as p1_sales,  max(case when product_id = %s then total_sales else 0 end) as p2_sales
        from sales_by_store
        group by store_id
    )
    select count(case when p1_sales > p2_sales then 1 end) as p1_outsells_p2_count, count(case when p2_sales > p1_sales then 1 end) as p2_outsells_p1_count
    from sales_comparison;
    """

    cursor.execute(query, (product_id1, product_id2, product_id1, product_id2))
    results = cursor.fetchall()

    # Format and display the results
    print(f"\nProduct Sales Comparison")
    print(f"{product_name1} (ID: {product_id1}) vs. {product_name2} (ID: {product_id2})")
    print("-" * 60)
    print("{:<40} {:<30}".format(
        f"Stores where '{product_name1}' sells more", 
        f"Stores where '{product_name2}' sells more"))
    print("-" * 60)

    if results:
        p1_outsells_p2_count = results[0][0]
        p2_outsells_p1_count = results[0][1]
        print("{:<40} {:<30}".format(p1_outsells_p2_count, p2_outsells_p1_count))
    else:
        print("No data available for comparison.")

    cursor.close()
    connection.close()

# Display products and stores for a specific customer
def productsAndStoresForCust():
    connection, cursor = connectToDatabase()
    
    # Get customer ID from user
    customer_id = input("Enter the customer ID: ").strip()

    # Query to find all products purchased by the customer and from which stores
    query = ("""
    select co.customer_id, so.store_id,p.product_id, p.product_name, ip.quantity, ip.price_at_purchase
    from customerOrder co
    join storeOrderedFrom so on co.order_id = so.order_id
    join itemsPurchased ip on ip.order_id = so.order_id
    join products p on ip.product_id = p.product_id
    where co.customer_id = %s;
    """)
    
    try:
        cursor.execute(query, (customer_id,))
        results = cursor.fetchall()

        if not results:
            print(f"No orders found for customer ID: {customer_id}")
        else:
            # Format and display the results
            print("\nCustomer Order Details")
            print("-" * 80)
            print("{:<12} {:<10} {:<12} {:<25} {:<10} {:<18}".format(
                "Customer ID", "Store ID", "Product ID", "Product Name", "Quantity", "Price Paid"
            ))
            print("-" * 80)
            for row in results:
                customer_id, store_id, product_id, product_name, quantity, price = row
                print("{:<12} {:<10} {:<12} {:<25} {:<10} ${:<17.2f}".format(
                    customer_id, store_id, product_id, product_name, quantity, price
                ))
    except mysql.connector.Error as err:
        print(f"Error retrieving data: {err}")
    finally:
        cursor.close()
        connection.close()

# Display order history for a specific customer
def custOrderHistory():
    connection, cursor = connectToDatabase()
    
    # Get customer ID from user
    customer_id = input("Enter the customer ID to view their order history: ").strip()

    # Query to get all order dates for the customer
    orderHistoryQuery = '''
    select c.customer_id, o.order_date
    from customerOrder as c
    left outer join orders as o on c.order_id = o.order_id
    where c.customer_id = %s;
    '''
    
    try:
        cursor.execute(orderHistoryQuery, (customer_id,))
        results = cursor.fetchall()

        if not results:
            print(f"No orders found for customer ID: {customer_id}")
        else:
            # Format and display the results
            print("\nOrder History")
            print("-" * 40)
            print("{:<15} {:<20}".format("Customer ID", "Order Date"))
            print("-" * 40)
            for customer_id, order_date in results:
                formatted_date = order_date.strftime("%Y-%m-%d") 
                print("{:<15} {:<20}".format(customer_id, formatted_date))
    except mysql.connector.Error as err:
        print(f"Error retrieving order history: {err}")
    finally:
        cursor.close()
        connection.close()

# Displaying the number of orders placed by each customer
def numOrdersByCust():
    connection, cursor = connectToDatabase()

    # Query to count orders by customer
    query = ("""
    select co.customer_id, count(co.order_id) as total_orders
    from customerOrder co
    group by co.customer_id
    order by total_orders desc;
    """)
    
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Formatting and displaying the results
    print("\nNumber of Orders by Customer")
    print("-" * 35)
    print("{:<15} {:<15}".format("Customer ID", "Total Orders"))
    print("-" * 35)    
    for customer_id, total_orders in results:
        print("{:<15} {:<15}".format(customer_id, total_orders))

# update inventory function
def updateInventory(product_id, amount_purchased):

    connection, cursor = connectToDatabase()

    # find the quantity of the product using product_id
    cursor.execute(
        f'''select quantity from storeInventory 
        where store_id = 1 
        and product_id = {product_id}; ''')

    currentquantity = cursor.fetchone()

    if currentquantity is None:
        print(f"No inventory record found for product_id {product_id} in online store")
    else:
        current_quantity = currentquantity[0]  # extract the integer
        
        new_quantity = current_quantity - amount_purchased
    
        cursor.execute(f"UPDATE storeInventory SET quantity = {new_quantity} WHERE store_id = 1 and product_id = {product_id};")

        connection.commit()

# Displaying welcome message for all users
def welcome():
    print("Welcome to Zara! We are so happy you are choosing us! We wish you the best experience working with our database!")

# Displaying welcome message for customers
def welcomeCust():
    print("We are so glad you are choosing to shop at Zara! We hope you have a magnificent, easy time purchasing from us!")

# Displaying welcome message for managers
def welcomeManager():
    print("Welcome to our database! We hope your experience as a manager has been magnificent!")
    
# Displaying main menu for user type selection
def displayPersonMenu():
    print("\nWelcome! Are you a customer or a store manager of Zara?")
    print("1 -- Customer")
    print("2 -- Store Manager")
    print("3 -- Exit\n")  

# Displaying menu for customer type
def displayCustomerMenu():
    print("\nAre you part of our membership rewards program?")
    print("1 -- Yes")
    print("2 -- No")
    print("3 -- Exit\n")

# Displaying menu for member customers
def displayCustMemberMenu():
    print("\nWhat would you like to do today?")
    print("1 -- Place an order")
    print("2 -- View your order details")
    print("3 -- Add a new payment method")
    print("4 -- Change your personal information")
    print("5 -- Return to main menu")
    print("6 -- Exit\n")

# Displaying menu for guest customers
def displayCustGuesMenu():
    print("\n1 -- Create an account")
    print("2 -- Place an order as a guest")
    print("3 -- Return to main menu")
    print("4 -- Exit\n")

# Displaying menu for store managers
def displayManagerMenu():
    print("\n1 -- View store and member trends")
    print("2 -- View overall Zara trends")
    print("3 -- Add a new product to a warehouse and store")
    print("4 -- Add inventory to the warehouse")
    print("5 -- Add inventory to the store")
    print("6 -- Go back")
    print("7 -- Exit\n")

# Displaying menu for store trends options
def displayStoreTrendsMenu():
    print("\n1 -- Identify the top 20 selling products at a specific store")
    print("2 -- For a given member, the products they bought and what stores they bought them from")
    print("3 -- Order history for a specific member")
    print("4 -- Frequent shopper data, amount of orders per customer")
    print("5 -- Go back")
    print("6 -- Exit\n")

# Displaying menu for Zara trends options
def displayZaraTrendsMenu():
    print("\n1 -- Identify the top 20 selling products overall")
    print("2 -- Identify the top 20 selling products by state")
    print("3 -- Identify the stores with the most sales so far this year")
    print("4 -- Identify the products that are most often bought together")
    print("5 -- Identify the products that outsell other products")
    print("6 -- Go back")
    print("7 -- Exit\n")

# Getting user menu choice with validation
def getMenuChoice(max_choice):
    while True:
        try:
            menu_choice = input("\nPlease enter the number here: ")
            print('\n')
            menu_num = int(menu_choice)
            if 0 <= menu_num <= max_choice:
                return menu_num
            else:
                print(f"Please enter a number between 0 and {max_choice}.")
        except ValueError:
            print("\nInvalid input. Please enter a valid integer.\n")

# Getting store ID choice with validation
def getStoreChoice(max_choice):
      while True:
        try:
            store_choice = input("\nPlease enter the store id here: ")
            store_num = int(store_choice)
            if 0 <= store_num <= max_choice:
                return store_num
            else:
                print(f"Please enter a number between 0 and {max_choice}.")
        except ValueError:
            print("\nInvalid input. Please enter a valid integer.")


def main():
    connection, cursor = connectToDatabase()
    welcome()
    customer_id = None 
    try:
        while True:
            displayPersonMenu()
            person_choice = getMenuChoice(3)
            if person_choice == 3:
                print("Exiting the application. Thank you!")
                sys.exit()
            elif person_choice == 1: 
                welcomeCust()
                while True:
                    displayCustomerMenu()
                    customer_menu_choice = getMenuChoice(3)
                    if customer_menu_choice == 3:
                        print("Exiting the application. Thank you!")
                        sys.exit()
                    elif customer_menu_choice == 1:  
                        customer_id = input("Enter your customer ID: ")
####### working on checking customer
                        checkcustomer = (f'''select customer_id from members where customer_id = {customer_id}''')
                        cursor.execute(checkcustomer)
                        checkresults = cursor.fetchall()

                        if checkresults:
                            print(f"Welcome back, customer ID: {customer_id}!")
                            while True:
                                displayCustMemberMenu()
                                member_choice = getMenuChoice(6)
                                if member_choice == 6:
                                    print("Exiting the application. Thank you!")
                                    sys.exit()
                                elif member_choice == 5:
                                    break 
                                elif member_choice == 1:
                                    placeOrderMember(customer_id)
                                elif member_choice == 2:
                                    viewOrderDetails(customer_id)
                                elif member_choice == 3:
                                    addPaymentMethod(customer_id)
                                elif member_choice == 4:
                                    changePersonalInformation(customer_id)
                        else:
                            print(f"\nCustomer ID {customer_id} is not in our system as a member.")
                            break
                    elif customer_menu_choice == 2:  
                        while True:
                            displayCustGuesMenu()
                            guest_choice = getMenuChoice(4)
                            if guest_choice == 4:
                                print("Exiting the application. Thank you!")
                                sys.exit()
                            elif guest_choice == 3:
                                break
                            elif guest_choice == 1:
                                createAccount()
                            elif guest_choice == 2:
                                placeOrderGuest(customer_id)
            elif person_choice == 2: 
                welcomeManager()
                while True:
                    displayManagerMenu()
                    manager_choice = getMenuChoice(7)
                    if manager_choice == 7:
                        print("Exiting the application. Thank you!")
                        sys.exit()
                    elif manager_choice == 6:
                        break 
                    elif manager_choice == 1:
                        while True:
                            displayStoreTrendsMenu()
                            store_trends_choice = getMenuChoice(6)
                            if store_trends_choice == 6:
                                print("Exiting the application. Thank you!")
                                sys.exit()
                            elif store_trends_choice == 5:
                                break
                            elif store_trends_choice == 1:
                                viewStoreTrends(store_trends_choice)
                            elif store_trends_choice == 2:
                                productsAndStoresForCust()
                            elif store_trends_choice == 3:
                                custOrderHistory()
                            elif store_trends_choice == 4:
                                numOrdersByCust()
                    elif manager_choice == 2:
                        viewZaraTrends()
                    elif manager_choice == 3:
                        addProductToWarehouseAndStore()
                    elif manager_choice == 4:
                        addInventoryToWarehouse()
                    elif manager_choice == 5:
                        addInventoryToStore()
    except SystemExit:
        print("Have a nice day!")

if __name__ == "__main__":
    main()

Welcome to Zara! We are so happy you are choosing us! We wish you the best experience working with our database!

Welcome! Are you a customer or a store manager of Zara?
1 -- Customer
2 -- Store Manager
3 -- Exit




Please enter the number here:  1




We are so glad you are choosing to shop at Zara! We hope you have a magnificent, easy time purchasing from us!

Are you part of our membership rewards program?
1 -- Yes
2 -- No
3 -- Exit




Please enter the number here:  1


Enter your customer ID:  6


...


These are the payment methods we have on file for your account:

 Credit Cards:
Payment ID   Card Number         
-----------------------------------
4            5989357044230593    
7            1879729072090049    
11           3561446548009337    

 Gift Cards on File:
Payment ID   Gift Card Number    
-----------------------------------
19           7284936376315810    
22           7595189922874214    



Please input what payment method you would like to use (enter payment_id):  4



Order placed successfully!

🧾 Your order ID is: 21



Press Enter to continue. 



What would you like to do today?
1 -- Place an order
2 -- View your order details
3 -- Add a new payment method
4 -- Change your personal information
5 -- Return to main menu
6 -- Exit

